# CEE6501 — Lecture 5.1

## Extending the DSM to 3D


## Learning Objectives

By the end of this lecture, you will be able to:

- Extend the Direct Stiffness Method (DSM) from 2D to **3D trusses**
- Write the **3D truss element** stiffness in local and global form
- State the **support constraints** needed to prevent rigid body motion in 3D


## Agenda

1. DSM in 3D: DOFs, rotations, and supports
2. Go over Assignment #3, where this will be implemented

## Part 1 — Extending the DSM to 3D Trusses

> Same DSM workflow, but with 3 translational DOFs per node and a 3D orientation.


### Degrees of freedom in 3D

For a 3D truss node:

$$
\mathbf{d}_i = \begin{bmatrix} u_{xi} & u_{yi} & u_{zi} \end{bmatrix}^T,
\quad
\mathbf{d}_j = \begin{bmatrix} u_{xj} & u_{yj} & u_{zj} \end{bmatrix}^T
$$

Element displacement vector in global coordinates:

$$
\mathbf{d}_e = \begin{bmatrix}
u_{xi} & u_{yi} & u_{zi} & u_{xj} & u_{yj} & u_{zj}
\end{bmatrix}^T  \equiv \text{DOFs }[1,2,3,4,5,6]
$$


### 3D direction cosines

Let the element connect nodes $i$ and $j$ with coordinates
$ (x_i,y_i,z_i)$ and $(x_j,y_j,z_j)$.

$$
L = \sqrt{(x_j-x_i)^2 + (y_j-y_i)^2 + (z_j-z_i)^2}
$$

$$
l = \frac{x_j-x_i}{L},\quad m = \frac{y_j-y_i}{L},\quad n = \frac{z_j-z_i}{L}
$$

These are the direction cosines of the element axis in global coordinates.


### 3D Truss Element Matrices

We will write:
- the **local** element stiffness $\mathbf{k}_{local}$ (6×6)
- the **transformation** matrix $\mathbf{T}$ (6×6)
- the **global** element stiffness $\mathbf{k}_{global}$ (6×6)


### Local stiffness matrix (6×6)

A truss element carries **axial force only** and therefore has stiffness only in the direction of its axis.

In a local coordinate system where the element axis is the local $x'$ direction, only the axial DOFs are active.

Then the 6×6 local stiffness is:

$$
\mathbf{k}_{local} = \frac{EA}{L}
\begin{bmatrix}
1 & 0 & 0 & -1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
-1 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$

Only axial coupling appears; transverse DOFs have zero stiffness in a truss model.


### Transformation Matrix: Local axes and direction cosines

A 3D truss element carries **axial force only**, but we still need a consistent mapping between
global and local translation components.

Let the element run from node $i$ to node $j$ with coordinates $(x_i,y_i,z_i)$ and $(x_j,y_j,z_j)$.

Define the element differences and length:

$$
\Delta x = x_j-x_i,\quad \Delta y = y_j-y_i,\quad \Delta z = z_j-z_i,
$$

$$
L = \sqrt{\Delta x^2 + \Delta y^2 + \Delta z^2}.
$$

The **axial direction cosines** (local $x'$ axis) are:

$$
l = \frac{\Delta x}{L},\quad m = \frac{\Delta y}{L},\quad n = \frac{\Delta z}{L}.
$$


### Why we need *two more* directions in 3D

In 2D, specifying the element axis automatically defines the perpendicular direction.
In 3D, the axis $(l,m,n)$ alone does **not** uniquely define a coordinate system: the element can still
rotate about its own axis.

So we complete an orthonormal local basis:
- $\hat{\mathbf e}_{x'}$ along the member (given by $(l,m,n)$)
- $\hat{\mathbf e}_{y'}$ transverse
- $\hat{\mathbf e}_{z'}$ transverse

These transverse directions do **not** add stiffness in a truss (still axial-only), but they make the
transformation well-defined.


### Constructing $\hat{\mathbf e}_{y'}$ and $\hat{\mathbf e}_{z'}$

Pick a reference vector $\mathbf a$ that is **not** parallel to the bar axis. A simple robust rule:

$$
\mathbf a =
\begin{cases}
\begin{bmatrix}0\\0\\1\end{bmatrix}, & |n|<0.9\\
\begin{bmatrix}0\\1\\0\end{bmatrix}, & |n|\ge 0.9 \quad \text{(bar nearly vertical)}
\end{cases}
$$



Define the local unit vectors:

$$
\hat{\mathbf e}_{x'} = \begin{bmatrix}l\\m\\n\end{bmatrix}
$$

$$
\hat{\mathbf e}_{y'} = \frac{\mathbf a \times \hat{\mathbf e}_{x'}}{\|\mathbf a \times \hat{\mathbf e}_{x'}\|}
$$

$$
\hat{\mathbf e}_{z'} = \hat{\mathbf e}_{x'} \times \hat{\mathbf e}_{y'}
$$

Write the transverse direction cosines as:

$$
\hat{\mathbf e}_{y'} = \begin{bmatrix}l_y\\m_y\\n_y\end{bmatrix}\qquad
\hat{\mathbf e}_{z'} = \begin{bmatrix}l_z\\m_z\\n_z\end{bmatrix}
$$

**Meaning:** $l_y$ is the $x$-component of the local $y'$ axis, $m_y$ is the $y$-component, etc.


### 3×3 rotation matrix $\mathbf R$ (direction cosines)

Collect the three local unit vectors (written in global components) into:

$$
\mathbf R =
\begin{bmatrix}
l & m & n\\
l_y & m_y & n_y\\
l_z & m_z & n_z
\end{bmatrix}
$$

By construction, the basis is orthonormal, so $\mathbf R\mathbf R^T = \mathbf I$.


## 6×6 transformation matrix $\mathbf T$

Same definition as in 2D:

$$
\mathbf d_{local} = \mathbf T\,\mathbf d_{global}
$$

with the block-diagonal transformation:

$$
\mathbf T =
\begin{bmatrix}
\mathbf R & \mathbf 0\\
\mathbf 0 & \mathbf R
\end{bmatrix}
$$

Explicitly (in direction cosines):

$$
\mathbf T=
\begin{bmatrix}
l & m & n & 0 & 0 & 0\\
l_y & m_y & n_y & 0 & 0 & 0\\
l_z & m_z & n_z & 0 & 0 & 0\\
0 & 0 & 0 & l & m & n\\
0 & 0 & 0 & l_y & m_y & n_y\\
0 & 0 & 0 & l_z & m_z & n_z
\end{bmatrix}
$$


## Global element stiffness (6×6)

Compute:

$$
\mathbf{k}_{global} = \mathbf{T}^T\,\mathbf{k}_{local}\,\mathbf{T}
$$

For a 3D truss element, the result can be written directly as:

$$
\mathbf{k}_{global} = \frac{EA}{L}
\begin{bmatrix}
 l^2 & lm  & ln  & -l^2 & -lm  & -ln  \\
 lm  & m^2 & mn  & -lm  & -m^2 & -mn  \\
 ln  & mn  & n^2 & -ln  & -mn  & -n^2 \\
-l^2 & -lm & -ln &  l^2 &  lm  &  ln  \\
-lm  & -m^2& -mn &  lm  &  m^2 &  mn  \\
-ln  & -mn & -n^2&  ln  &  mn  &  n^2
\end{bmatrix}
$$

This is the standard 3D truss element stiffness in global coordinates.

Even though $\mathbf T$ is a full 3D rotation, a truss remains axial-only. That is why the closed-form $\mathbf k_{global}$ only contains $l,m,n$ terms.



In [2]:
import sympy as sp

# --- symbols ---
EA_L = 1
l, m, n = sp.symbols("l m n", real=True)
ly, my, ny = sp.symbols("l_y m_y n_y", real=True)
lz, mz, nz = sp.symbols("l_z m_z n_z", real=True)

# --- T matrix (6x6) ---
T = sp.Matrix([
    [l,  m,  n,  0,  0,  0],
    [ly, my, ny, 0,  0,  0],
    [lz, mz, nz, 0,  0,  0],
    [0,  0,  0,  l,  m,  n],
    [0,  0,  0,  ly, my, ny],
    [0,  0,  0,  lz, mz, nz],
])

# --- local stiffness (6x6) ---
k_local = EA_L * sp.Matrix([
    [ 1, 0, 0, -1, 0, 0],
    [ 0, 0, 0,  0, 0, 0],
    [ 0, 0, 0,  0, 0, 0],
    [-1, 0, 0,  1, 0, 0],
    [ 0, 0, 0,  0, 0, 0],
    [ 0, 0, 0,  0, 0, 0],
])

In [3]:
# --- global stiffness via transformation ---
k_global = sp.simplify(T.T * k_local * T)

# Pretty output (in notebook)
sp.pprint(k_global)

⎡  2                 2             ⎤
⎢ l    l⋅m   l⋅n   -l    -l⋅m  -l⋅n⎥
⎢                                  ⎥
⎢        2                 2       ⎥
⎢l⋅m    m    m⋅n   -l⋅m  -m    -m⋅n⎥
⎢                                  ⎥
⎢              2                 2 ⎥
⎢l⋅n   m⋅n    n    -l⋅n  -m⋅n  -n  ⎥
⎢                                  ⎥
⎢  2                 2             ⎥
⎢-l    -l⋅m  -l⋅n   l    l⋅m   l⋅n ⎥
⎢                                  ⎥
⎢        2                 2       ⎥
⎢-l⋅m  -m    -m⋅n  l⋅m    m    m⋅n ⎥
⎢                                  ⎥
⎢              2                 2 ⎥
⎣-l⋅n  -m⋅n  -n    l⋅n   m⋅n    n  ⎦


### Supports and stability in 3D

A free rigid body in **3D** has **6 rigid body modes**:
- 3 translations
- 3 rotations

In a **3D truss model**, only **translations** are included as DOFs, but the structure
can still undergo rigid body motion unless enough nodal translations are constrained.

- In **2D trusses**, at least **3 independent translational restraints** are required
  to prevent rigid body motion.
- In **3D trusses**, at least **6 independent translational restraints** are required.

Typical sufficient constraint patterns include:
- Fixing all three translations at one node, two translations at a second node,
  and one translation at a third node (provided the geometry itself is not degenerate)

### What does “degenerate geometry” mean?

A constraint set is **degenerate** if it does *not* eliminate all rigid body motion,
even though the **number** of restrained DOFs is sufficient.

In other words: you may have *six* restraints in 3D, but their **locations or directions**
do not block all translations and rotations.

Examples of degenerate support layouts:
- All constrained nodes lie on a **single line** → rotation about that line remains
- All constrained nodes lie in a **single plane**, allowing rotation out of the plane
- Constrained directions are **collinear** or redundant
- A “one-DOF” restraint acts in a direction that does not resist any remaining rigid motion

**Key idea:**  
It is not just *how many* DOFs you constrain, but *where* and *in what directions*.

If constraints are degenerate, the global stiffness matrix $\mathbf{K}$ is **singular**.



## Part 2 — Discuss Assignment

### Quick Note — Install Plotly

You will need **Plotly** installed in your **conda environment** for the 3D / interactive truss visualization in Assignment 2.

Please make sure it is installed **in the same environment** you are using for Jupyter.



### Install from Integrated Terminal (VS-code) or External Terminal

List all available environments

```bash
conda env list
```

Activate your environment first:

```bash
conda activate <YOUR_ENV_NAME>
```

Then install:
```bash
conda install plotly
```


Check version:

```bash
conda list plotly
```

You should see something like:

```
plotly    6.5.2    pyhd8ed1ab_0
```

The version **should be 6.5.2**.


### Why Plotly? (Interactive 3D Example)

Plotly is powerful because it gives you:

- ✅ Interactive 3D rotation (drag with mouse)
- ✅ Zoom in/out
- ✅ Hover tooltips
- ✅ Clean visuals without extra setup
- ✅ Works in Jupyter, VS Code, Colab, etc.

For structural mechanics, this is extremely useful when visualizing:
- 3D trusses  
- Deformed shapes  
- Force magnitudes  
- Node labels  

In [4]:
import plotly.graph_objects as go
import numpy as np

# Simple 3D spiral
t = np.linspace(0, 6*np.pi, 200)
x = np.cos(t)
y = np.sin(t)
z = t

In [5]:
fig = go.Figure(
    go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='lines',
        line=dict(width=5)
    )
)

fig.update_layout(
    title="Interactive 3D Plot (Drag to Rotate)",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    )
)

fig.show()

### Screenshot of Plot

Can't show dynamic plot on a static slide. 

See lecture slide code for dynamic implementation

![Example truss bridge](assets/l1_newplot.png)